# Setup

 Import standard modules

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
os.chdir('/home/chris/projects/tidal_flat_0d')

In [2]:
from src import features, models
from pyprojroot import here
root = here()
sys.path.append(str(root))
wdir = root / "data" / "interim"

Import project modules from src folder

In [3]:
tides = features.load_tide(wdir, "tides.feather")
tides = tides - (np.mean(tides) + 0.6)
# tides = tides.iloc[:int(len(tides) * 0.35)]

In [6]:
tides.index = np.arange(0,len(tides))

In [8]:
tides

0          -0.863023
1          -0.863186
2          -0.863348
3          -0.863511
4          -0.863674
              ...   
31535995   -1.359709
31535996   -1.359471
31535997   -1.359234
31535998   -1.358997
31535999   -1.358759
Length: 31536000, dtype: float64

In [11]:
T = models.Tides(tides, 1.15, params=models.Params(org_rate=2e-4, comp_rate=-4e-3,
    sub_rate=-3e-3,))

In [15]:
T.df.

,tide_elev,land_elev
0,-0.863023,1.1500
1,-0.863186,1.1500
2,-0.863348,1.1500
3,-0.863511,1.1500
4,-0.863674,1.1500
...,...,...
31535995,-1.359709,1.1428
31535996,-1.359471,1.1428
31535997,-1.359234,1.1428
31535998,-1.358997,1.1428


In [107]:
import time
from scipy.interpolate import InterpolatedUnivariateSpline
i = 100000
r = 1000
rr = int(r / 2)
arr = np.arange(i-rr,i+rr, 1)
fun = InterpolatedUnivariateSpline(x=T.df.index.values, y=T.df.tide_elev.values)
ts = np.empty(0)
for t in range(0,10):
    s = time.perf_counter()
    fun(arr)
    ts = np.append(ts, time.perf_counter() - s)
t1 = np.mean(ts)
ts = np.empty(0)
for t in range(0,10):
    s = time.perf_counter()
    fun2 = InterpolatedUnivariateSpline(x=T.df.index.values[i-rr:i+rr], y=T.df.tide_elev.values[i-rr:i+rr])
    fun2(arr)
    ts = np.append(ts, time.perf_counter() - s)
t2 = np.mean(ts)
times = {'t1': t1, 't2': t2}
diff = eval(max(times)) - eval(min(times))
print("{} is faster on average by {:.2e} s per inundation cycle.".format(min(times), diff))
print("This amounts to ~{:.2f} s per year".format(diff * 185))

t1 is faster on average by 1.52e-04 s per inundation cycle.
This amounts to ~0.03 s per year


In [ ]:
from src import models

sim = models.Simulation(
    tide_ts=tides,
    land_elev_init=1.15,
    conc_init=0.0,
    conc_bound=1.0,
    grain_diam=3.5e-5,
    grain_dens=2.65e3,
    bulk_dens=900,
    org_rate=2e-4,
    comp_rate=-4e-3,
    sub_rate=-3e-3,
    slr=0.005,
    solver_method="DOP853",
    runs=2,
    min_depth=0.0015,
    dense_output=False,
)
# sim.run_all()

In [ ]:
sim.tides.df

In [ ]:
sim.results.inundations[-1].pos_start

In [ ]:
sim.tides.subset(start=0, end=10)

In [ ]:
import time
def aggrade_fd(times, tide_fun, init_land_elev, bound_conc, settle_rate, bulk_dens,):
    start_time = time.perf_counter()
    land_elev = np.array([init_land_elev, init_land_elev])
    conc = np.array([0.0, 0.0])
    depth = np.array([tide_fun(0) - init_land_elev])
    status = 0
    index = np.arange(0, len(times))
    dt = times[1] - times[0]
    for i in index[1:-1]:
        t = times[i]
        depth = np.append(depth, tide_fun(t) - land_elev[i])
        if depth[i] < 0.0015:
            conc = np.append(conc, 0.0)
            land_elev = np.append(land_elev, land_elev[-1])
            continue

        if (depth[i] - depth[i-1]) > 0:
            delta_conc = - settle_rate * conc[i] / depth[i] * dt - 1 / depth[i] * (conc[i] - bound_conc) * (depth[i] - depth[i-1])
        elif (depth[i] - depth[i-1]) <= 0:
            if status == 0:
                slack = times[i]
                status = 1
            delta_conc = - settle_rate * conc[i] / depth[i] * dt
            
        if delta_conc < 0 and abs(delta_conc) > conc[i]:
            print("Extracted too much")
            delta_conc = - conc[i]
            
        conc = np.append(conc, conc[i] + delta_conc)
        delta_land_elev = settle_rate * conc[i] / bulk_dens * dt
        land_elev = np.append(land_elev, land_elev[i] + delta_land_elev)
        
    depth = np.append(depth, tide_fun(i+1) - land_elev[i])
    print("Aggradation:           {:.2e} m".format(land_elev[-1] - land_elev[0]))
    print("Degradation:           {:.2e} m".format(0.0))
    print("Elevation change:      {:.2e} m".format(land_elev[-1] - land_elev[0]))
    print("Final elevation:       {:.4f} m".format(land_elev[-1]))
    print("Runtime:               {}".format(time.strftime("%H:%M:%S", time.gmtime(time.perf_counter() - start_time))))

    return slack, np.stack((times, depth, conc, land_elev))